# Exploratory Analysis

## Team Members: Joshua Holman, Josue Cota


In [1]:
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

# Data dictionary project.csv contains descriptions of each data val
hsd = pd.read_csv("Health Sciences Data File New (project).csv")

hsd = hsd.dropna(axis=1)
hsd

,Idnum,Date,Sex,Age,Ht,Wt,RF 2,RF 3,RF 4,RF 5,...,DBP,HR rest,Stages,PL 1,HR 1,RPE 1,PL 2,HR 2,RPE 2,FFTotal
0,M0001,2002-2,M,19,65.0,195.4,5,1,1,0,...,86,72,3,50,122,12,100,148,13.0,25
1,M0002,2002-2,M,19,68.5,201.4,5,2,2,0,...,72,72,3,50,107,8,125,131,12.0,32
2,M0003,2002-2,M,21,69.8,133.8,5,2,6,0,...,68,72,3,50,119,10,100,144,17.0,39
3,M0004,2002-2,M,18,66.0,148.0,5,1,6,0,...,78,72,3,50,108,7,125,157,14.0,37
4,M0005,2002-2,M,23,67.0,148.7,5,2,6,2,...,70,72,3,50,118,6,100,128,7.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6271,F3612,2013-4,F,18,68.0,193.8,1,3,1,0,...,76,72,3,30,143,9,50,158,9.0,21
6272,F3613,2013-4,F,19,64.0,106.0,1,2,1,0,...,60,72,3,30,144,12,50,150,12.0,37
6273,F3614,2013-4,F,23,67.0,155.8,1,3,1,0,...,60,72,4,50,85,6,100,99,8.0,51
6274,F3615,2013-4,F,18,68.0,152.6,1,3,1,0,...,60,72,4,50,91,6,100,111,10.0,54


In [ ]:
import matplotlib.pyplot as plt

columns = ["Wt", "Ht", "HR rest", "SBP", "DBP"]


plots = []
for column in columns:
    p = (ggplot(hsd)
         + geom_histogram(aes(x=hsd[column]),  color="white")
         + labs(title="{column}, frequency distribution", x=column, y="Frequency")
        )
    plots.append(p)

# Weight
plots[0]


Relatively normal distribution and the data seems to match the expected curve.

In [3]:
# Height
plots[1]

Height: Somewhat normally distributed, random dips in certain heights but otherwise fine

In [4]:
# HR Rest
plots[2]

HR REST: Expected outcome, not great in histogram form

In [5]:
# SBP
plots[3]

SBP (Systolic): Mostly normally distributed. Expected to have highs at normal Systolic Pressures

In [6]:
# DBP
plots[4]

DBP (Diastolic): Normally Distributed with normal high counts at average Diastolic rates, few outliers. 

Overall, these 5 data variables have good data. Reasons for Diastolic and Systolic having several high points might be due to averages between healthier vs less healthy people or weight differences perhaps.